In [41]:
from pyspark import SparkConf, SparkContext
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [42]:
conf = SparkConf().setMaster('local[4]').setAppName('category-review-average')
sc = SparkContext(conf=conf)

In [43]:
folder_path = '/Users/lifelog/_PML/fc_spark/data'
file_name = 'restaurant_reviews.csv'

lines = sc.textFile(f'file:///{folder_path}/{file_name}') # line RDD (Action 되기 까지 실행 X)
lines

file:////Users/lifelog/_PML/fc_spark/data/restaurant_reviews.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [46]:
lines.collect() # 간단히 확인할 수 있는 Action API

['id,item,cateogry,reviews',
 '0,짜장면,중식,125',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32',
 '3,떡볶이,분식,534',
 '4,라멘,일식,223   ',
 '5,돈가스,일식,52',
 '6,우동,일식,12',
 '7,쌀국수,아시안,312',
 '8,햄버거,패스트푸드,12',
 '9,치킨,패스트푸드,23']

In [57]:
header_row = lines.first()
usecols_lines = lines.filter(lambda x: x != header_row)
usecols_lines_pair = usecols_lines.map(lambda x: (x.split(',')[2], int(x.split(',')[3])))

In [58]:
usecols_lines_pair.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [93]:
usecols_lines_pair.groupByKey().mapValues(lambda x: sum(x)).collect()

[('분식', 566), ('아시안', 312), ('패스트푸드', 35), ('중식', 360), ('일식', 287)]

In [62]:
categoryReviewsCount = usecols_lines_pair.mapValues(lambda v: (v, 1))
categoryReviewsCount.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [79]:
categoryReviewsCount.reduceByKey(lambda x, y: x[0]).collect()

[('분식', 32), ('아시안', (312, 1)), ('패스트푸드', 12), ('중식', 125), ('일식', 223)]

In [66]:
# reduceByKey 2개의 값이 전달됨!!!!
reduced = categoryReviewsCount.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
reduced.collect()

[('분식', (566, 2)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2)),
 ('중식', (360, 2)),
 ('일식', (287, 3))]

In [67]:
reduced.mapValues(lambda x: x[0] / x[1]).collect()

[('분식', 283.0),
 ('아시안', 312.0),
 ('패스트푸드', 17.5),
 ('중식', 180.0),
 ('일식', 95.66666666666667)]